In [ ]:
import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer  # Til tekstbehandling
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report, f1_score



In [ ]:
df=pd.read_csv('cleanedStemmedCATbbc.csv') 

In [ ]:
df['broad_category'].value_counts()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['content'], df['broad_category'], test_size=0.2, random_state=42)

X_train_vec og X_test_vec indeholder tekst nemlig 'content'.
Ved brug af modulet 'TfidfVectorizer' kan vi omdanne hver tekst til en række af vægtede tal baseret på TF-IDF (Term Frequency-Inverse Document Frequency).
- Term Frequency (TF): Hvor ofte et ord optræder i en given tekst.
- Inverse Document Frequency (IDF): Hvor sjældent ordet optræder på tværs af alle dokumenter – jo sjældnere, desto vigtigere.

In [ ]:
#Vektoriser tekstdata (TF-IDF)
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [ ]:
import gensim.downloader as api
model = api.load("word2vec-google-news-300")


In [ ]:
import numpy as np

# Funktion til at konvertere tekst til Word2Vec-vektorer
def text_to_w2v_vector(text, model, vector_size=300):
    words = text.split()  # Split teksten i ord
    word_vectors = [model[word] for word in words if word in model]
    if len(word_vectors) == 0:
        return np.zeros(vector_size)  # Hvis ingen ord findes, returnér nul-vektor
    return np.mean(word_vectors, axis=0)  # Gennemsnit af alle ordvektorer

In [ ]:
# Antag at X_train og X_test indeholder tekst
X_train_vec = np.array([text_to_w2v_vector(text, model) for text in X_train])
X_test_vec = np.array([text_to_w2v_vector(text, model) for text in X_test])


In [ ]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score

# Træn SVM-model
model = LinearSVC(class_weight='balanced', C=0.8, dual=False, max_iter=10000)
model.fit(X_train_vec, y_train)

# Forudsig på testdata
y_pred = model.predict(X_test_vec)

# Evaluer præstation
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Model accuracy: {accuracy * 100:.2f}%")
print(f"Model F1-score: {f1:.2f}")